In [1]:
import numpy as np
import torch

In [2]:
bikes_np = np.loadtxt('../data/p1ch4/bike-sharing-dataset/hour-fixed.csv',
                      dtype=np.float32, delimiter=",", skiprows=1,
                      converters={1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_np)
bikes.shape, bikes

(torch.Size([17520, 17]),
 tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
          1.6000e+01],
         [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
          4.0000e+01],
         [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
          3.2000e+01],
         ...,
         [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
          9.0000e+01],
         [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
          6.1000e+01],
         [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
          4.9000e+01]]))

In [4]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [5]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [7]:
daily_bikes = daily_bikes.transpose(1,2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [11]:
first_day = bikes[:24].long()
first_day.shape, first_day[:,9]

(torch.Size([24, 17]),
 tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2]))

In [10]:
weather_onehot = torch.zeros(first_day.shape[0], 4)
weather_onehot.shape

torch.Size([24, 4])

In [12]:
weather_onehot.scatter_(dim=1, index=first_day[:,9].unsqueeze(1).long()-1, value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [13]:
bikes.shape, weather_onehot.shape

(torch.Size([17520, 17]), torch.Size([24, 4]))

In [15]:
bikes[:24].shape

torch.Size([24, 17])

In [18]:
torch.cat((bikes[:24], weather_onehot), 1).shape

torch.Size([24, 21])

In [20]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,        
                           daily_bikes.shape[2])
daily_weather_onehot.shape, daily_bikes.shape

(torch.Size([730, 4, 24]), torch.Size([730, 17, 24]))

In [22]:
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape, daily_weather_onehot

(torch.Size([730, 4, 24]),
 tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0.,

In [23]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
daily_bikes.shape, daily_bikes

(torch.Size([730, 21, 24]),
 tensor([[[1.0000e+00, 2.0000e+00, 3.0000e+00,  ..., 2.2000e+01,
           2.3000e+01, 2.4000e+01],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],
 
         [[2.5000e+01, 2.6000e+01, 2.7000e+01,  ..., 4.5000e+01,
           4.6000e+01, 4.7000e+01],
          [2.0000e+00, 2.0000e+00, 2.0000e+00,  ..., 2.0000e+00,
           2.0000e+00, 2.0000e+00],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          ...,
          [1.0000e+00, 1.0000e

In [26]:
daily_bikes[:,9,:]

tensor([[1., 1., 1.,  ..., 2., 2., 2.],
        [2., 2., 2.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [2., 2., 2.,  ..., 2., 2., 2.],
        [2., 2., 2.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [27]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

In [28]:
daily_bikes[:,9,:]

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.3333, 0.3333, 0.3333,  ..., 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [29]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)/(temp_max - temp_min))

In [30]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp)) / torch.std(temp))

In [31]:
daily_bikes[:,10,:]

tensor([[-1.3213, -1.4248, -1.4248,  ..., -0.4932, -0.4932, -0.1827],
        [-0.1827, -0.2862, -0.3897,  ..., -1.2178, -1.3213, -1.4248],
        [-1.4248, -1.5284, -1.5284,  ..., -1.6319, -1.8389, -1.6319],
        ...,
        [-1.3213, -1.3213, -1.3213,  ..., -1.1143, -1.2178, -1.2178],
        [-1.2178, -1.2178, -1.2178,  ..., -1.5284, -1.5284, -1.5284],
        [-1.6319, -1.6319, -1.7354,  ..., -1.2178, -1.2178, -1.2178]])

In [32]:
daily_bikes.shape

torch.Size([730, 21, 24])